# W207 Final Project
## Home Price Prediction
Mona Iwamoto, Michelle Kim, Alex Lau, Alex Nocella

In [23]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn.linear_model as linear_model
import sklearn.metrics as metrics

train_all = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train_all = pd.get_dummies(train_all)
train_all = train_all.fillna(train_all.mean())

print train_all.shape

# split train into train and dev
train = train_all[:1000]
dev = train_all[1000:]

# Split into input and output dataframes.
train_X, train_y = train.loc[:, train.columns != 'SalePrice'], train.loc[:, train.columns == 'SalePrice']
dev_X, dev_y = dev.loc[:, dev.columns != 'SalePrice'], dev.loc[:, dev.columns == 'SalePrice']

train_X = train_X[:train_X.shape[0]]
train_y = train_y.SalePrice

dev_X = dev_X[:dev_X.shape[0]]
dev_y = dev_y.SalePrice

print train_X.shape
print train_y.shape
print dev_X.shape
print dev_y.shape
print train.head()

(1460, 290)
(1000, 289)
(1000,)
(460, 289)
(460,)
   Id  MSSubClass  LotFrontage  LotArea  OverallQual  OverallCond  YearBuilt  \
0   1          60         65.0     8450            7            5       2003   
1   2          20         80.0     9600            6            8       1976   
2   3          60         68.0    11250            7            5       2001   
3   4          70         60.0     9550            7            5       1915   
4   5          60         84.0    14260            8            5       2000   

   YearRemodAdd  MasVnrArea  BsmtFinSF1          ...            \
0          2003       196.0         706          ...             
1          1976         0.0         978          ...             
2          2002       162.0         486          ...             
3          1970         0.0         216          ...             
4          2000       350.0         655          ...             

   SaleType_ConLw  SaleType_New  SaleType_Oth  SaleType_WD  \
0         

In [54]:
# Set up simplest linear regression model possible:
simple_LR = linear_model.LinearRegression()
simple_LR.fit(X = train_X.GrLivArea.values.reshape(-1, 1), y = train_y.values.reshape(-1, 1))

print("y-intercept: {:,.2f}".format(float(simple_LR.intercept_)))
print("GrLivArea coefficient: {:,.2f}".format(float(simple_LR.coef_)))

simple_LR_pred_labels = simple_LR.predict(dev_X.GrLivArea.values.reshape(-1, 1))
print("Root Mean Squared Error: {:,.2f}".format(metrics.mean_squared_error(dev_y, simple_LR_pred_labels) ** 0.5))

# Calculate R-squared values of model.
print("R-squared: {:.2f}".format(metrics.r2_score(dev_y, simple_LR_pred_labels)))

y-intercept: 8,283.58
GrLivArea coefficient: 115.19
Root Mean Squared Error: 59,977.54
R-squared: 0.40


In [22]:
#Maybe Use Decision Tree/Random Forests to Pick Best Features?
from sklearn.tree import DecisionTreeRegressor

tree=DecisionTreeRegressor(max_depth=3)
tree.fit(train_X,train_y)

#print tree.feature_importances_
features = np.where(tree.feature_importances_!=0)[0]
for i in features:
    print list(train_X)[i]

OverallQual
TotalBsmtSF
GrLivArea
ExterQual_Ex


In [61]:
#Write out the csv of predictions
output = dev_X.copy()
output['SalePrice']=simple_LR.predict(output.GrLivArea.values.reshape(-1,1))
output.to_csv('Output.csv',columns=['Id','SalePrice'],index=False)